In [1]:
dbutils.widgets.text("datasets", "")

In [2]:
datasets = dbutils.widgets.get("datasets")

In [3]:
%run ../Datasets/DatasetBase

In [4]:
from pyspark.sql.functions import lower, col
datasets = str(datasets).replace(' ','').lower().split(',')

bucket_write = dbutils.secrets.get(scope = "aws", key = "aws-bucket-name")
DatasetBase.createBucket(bucket_write)

complete_path = f'/mnt/{bucket_write}/raw/datasets'

datasetsDataFrame = DatasetBase.loadDatabase(bucket_write, complete_path)
display(datasetsDataFrame)

id,name,bucket_name,path,delimiter,extension,created_at
2,OrderStatuses,ifood-data-architect-test-source,status.json.gz,,json,2020-08-03T05:32:48.000Z
4,Restaurant,ifood-data-architect-test-source,restaurant.csv.gz,",",csv,2020-08-03T05:33:55.000Z
3,Consumer,ifood-data-architect-test-source,consumer.csv.gz,",",csv,2020-08-03T05:33:27.000Z
1,Order,ifood-data-architect-test-source,order.json.gz,,json,2020-08-03T05:32:10.000Z


In [5]:
sss

In [6]:
from datetime import datetime

for dataset in datasets:
    if datasetsDataFrame.filter(lower(col('name')) == dataset).count() == 1:
        _, name, bucket_name, path, delimiter, extension, _ = datasetsDataFrame.filter(lower(col('name')) == dataset).collect()[0]
        
        newDataset = DatasetBase(bucket_name, path, delimiter, extension)
        DatasetBase.createBucket(bucket_name)
        
        complete_path = f'/mnt/{bucket_name}/{path}'
        newDataset.setVariables()
        df = newDataset.loadDatabase(bucket_name, complete_path, extension)
        
        path_write = f'/mnt/{bucket_write}/raw/{name.lower()}/{datetime.now().strftime("%Y")}/{datetime.now().strftime("%m")}/{datetime.now().strftime("%d")}'
        DatasetBase.writeDataset(df, bucket_write, path_write)
        
        print(f'Dataset {name} carregado com Sucesso')
    else:
        print(f'Dataset {name} não cadastrado!!')

/mnt/ifood-data-architect-test-source/order.json.gz
Dataset Order carregado com Sucesso
/mnt/ifood-data-architect-test-source/status.json.gz
Dataset OrderStatuses carregado com Sucesso
Dataset Restaurant carregado com Sucesso
Dataset Consumer carregado com Sucesso

In [7]:
display(dbutils.fs.ls(f'/mnt/{bucket_write}/raw/'))

path,name,size
dbfs:/mnt/[REDACTED]/raw/consumer/,consumer/,0
dbfs:/mnt/[REDACTED]/raw/datasets/,datasets/,0
dbfs:/mnt/[REDACTED]/raw/order/,order/,0
dbfs:/mnt/[REDACTED]/raw/orderstatuses/,orderstatuses/,0
dbfs:/mnt/[REDACTED]/raw/restaurant/,restaurant/,0
